In [9]:

!pip -q install plotly ipywidgets

# (προαιρετικό, βοηθά widgets σε Colab)
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except Exception:
    pass


In [11]:
# Rotatable 3D & corrected Heatmap για f(x,y) = a(x - S1) + b(y - S2) στο Colab
# Περιοχή: x>0, y>0 • Είσοδοι: S1,S2 >= 0 (χωρίς άνω φράγμα)

import numpy as np
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go

# === Widgets ===
a_w  = widgets.FloatText(value=1.0, description='a', layout=widgets.Layout(width='200px'))
b_w  = widgets.FloatText(value=1.0, description='b', layout=widgets.Layout(width='200px'))
S1_w = widgets.FloatText(value=1.0, description='S1 (≥0)', layout=widgets.Layout(width='200px'))
S2_w = widgets.FloatText(value=1.0, description='S2 (≥0)', layout=widgets.Layout(width='200px'))

x_min_w = widgets.FloatText(value=0.1, description='x_min', layout=widgets.Layout(width='200px'))
x_max_w = widgets.FloatText(value=10.0, description='x_max', layout=widgets.Layout(width='200px'))
y_min_w = widgets.FloatText(value=0.1, description='y_min', layout=widgets.Layout(width='200px'))
y_max_w = widgets.FloatText(value=10.0, description='y_max', layout=widgets.Layout(width='200px'))

res_w   = widgets.IntSlider(value=80, min=20, max=400, step=10, description='Resolution', readout=True)
view_w  = widgets.ToggleButtons(options=['3D (rotatable)', 'Heatmap (2D)'], value='3D (rotatable)', description='View')

def _bounds(xmin, xmax, ymin, ymax):
    eps = 1e-9
    xmin = max(eps, float(xmin))
    ymin = max(eps, float(ymin))
    xmax = float(xmax); ymax = float(ymax)
    if not np.isfinite(xmax) or xmax <= xmin: xmax = xmin + 1.0
    if not np.isfinite(ymax) or ymax <= ymin: ymax = ymin + 1.0
    return xmin, xmax, ymin, ymax

def _grid(xmin, xmax, ymin, ymax, res):
    x = np.linspace(xmin, xmax, int(res))
    y = np.linspace(ymin, ymax, int(res))
    X, Y = np.meshgrid(x, y)  # shape: (ny, nx)
    return x, y, X, Y

def _zero_line_xy(a, b, S1, S2, xmin, xmax, ymin, ymax):
    # Επιστρέφει (x_line, y_line) για τη γραμμή f=0 μέσα στο παράθυρο αν υπάρχει
    if abs(b) > 1e-12:
        xs = np.linspace(xmin, xmax, 400)
        ys = S2 - (a/b)*(xs - S1)
        mask = (ys >= ymin) & (ys <= ymax)
        xs, ys = xs[mask], ys[mask]
        return xs, ys
    elif abs(a) > 1e-12:
        ys = np.linspace(ymin, ymax, 400)
        xs = np.full_like(ys, S1)
        return xs, ys
    else:
        # a=b=0: f ≡ 0 παντού → δεν χρειάζεται γραμμή
        return None, None

def plot_view(a, b, S1, S2, x_min, x_max, y_min, y_max, res, view):
    # Clamp/έλεγχοι
    a = float(a); b = float(b)
    S1 = max(0.0, float(S1)); S2 = max(0.0, float(S2))
    xmin, xmax, ymin, ymax = _bounds(x_min, x_max, y_min, y_max)

    x, y, X, Y = _grid(xmin, xmax, ymin, ymax, res)
    Z = a * (X - S1) + b * (Y - S2)
    vmax = np.max(np.abs(Z)); vmax = (1.0 if vmax == 0 else float(vmax))

    if view == '3D (rotatable)':
        fig = go.Figure()

        # 3D επιφάνεια με συμμετρική κλίμακα γύρω από 0
        fig.add_trace(go.Surface(
            x=X, y=Y, z=Z,
            cmin=-vmax, cmax=vmax,
            colorscale='RdBu',
            colorbar=dict(title='f(x,y)')
        ))

        # Γραμμή f=0 (πάνω στην επιφάνεια, z=0)
        x0, y0 = _zero_line_xy(a, b, S1, S2, xmin, xmax, ymin, ymax)
        if x0 is not None:
            fig.add_trace(go.Scatter3d(
                x=x0, y=y0, z=np.zeros_like(x0),
                mode='lines', line=dict(color='black', width=6),
                name='f=0'
            ))

        fig.update_layout(
            title='Rotatable 3D: f(x,y) = a(x − S1) + b(y − S2)',
            scene=dict(
                xaxis_title='x', yaxis_title='y', zaxis_title='f(x,y)',
                aspectmode='cube'
            ),
            margin=dict(l=0, r=0, t=40, b=0)
        )
        fig.show()

    else:
        # === CORRECTED HEATMAP ===
        # Χρησιμοποιούμε Heatmap (όχι Contour) και κεντράρουμε στο 0 με zmid=0
        fig = go.Figure()

        fig.add_trace(go.Heatmap(
            x=x, y=y, z=Z,
            zauto=False, zmin=-vmax, zmax=vmax, zmid=0,
            colorscale='RdBu',
            colorbar=dict(title='f(x,y)'),
            hovertemplate='x=%{x:.3g}<br>y=%{y:.3g}<br>f=%{z:.3g}<extra></extra>'
        ))

        # Γραμμή f=0 πάνω από το heatmap
        x0, y0 = _zero_line_xy(a, b, S1, S2, xmin, xmax, ymin, ymax)
        if x0 is not None and x0.size > 1:
            fig.add_trace(go.Scatter(
                x=x0, y=y0,
                mode='lines',
                line=dict(color='black', width=3),
                name='f=0',
                hoverinfo='skip'
            ))

        fig.update_layout(
            title='Heatmap (x>0, y>0): f(x,y) = a(x − S1) + b(y − S2)',
            xaxis_title='x', yaxis_title='y',
            yaxis_scaleanchor='x',  # ίση κλίμακα αξόνων
            margin=dict(l=0, r=0, t=40, b=0)
        )
        fig.show()

controls = widgets.HBox([
    widgets.VBox([a_w, b_w]),
    widgets.VBox([S1_w, S2_w]),
    widgets.VBox([x_min_w, x_max_w]),
    widgets.VBox([y_min_w, y_max_w]),
    widgets.VBox([res_w, view_w]),
])

out = widgets.interactive_output(
    plot_view,
    {
        'a': a_w, 'b': b_w, 'S1': S1_w, 'S2': S2_w,
        'x_min': x_min_w, 'x_max': x_max_w,
        'y_min': y_min_w, 'y_max': y_max_w,
        'res': res_w, 'view': view_w
    }
)

display(controls, out)


Output()